## Create and Destroy an Azure Kubernetes Service in Azure using Terraform

- An Azure Kubernetes Service (AKS) is a kubernetes cluster running in Azure.
  - Since an Azure Kubernetes Service (AKS) is an Azure resource, it must be placed in an Azure Resource Group.
  - Since an Azure Kubernetes Service (AKS) creates a Virtual Network, we also need to create a Network Watcher.
  - Since an Azure Kubernetes Service (AKS) needs to pull its images from a Registry, we also create an Azure Container Registry.
- This Terraform Project consists of the Terraform files listed below:

In [1]:
#!dir *.tf # use this on Windows
!ls *.tf

container-registry.tf  network-watcher.tf  resource-group.tf
kubernetes-cluster.tf  providers.tf	   variables.tf


## Terraform providers

- We are using the same Terraform proviers as before (i.e. the `azurerm` provider for Azure).

In [2]:
#!type providers.tf # use this on Windows
!cat providers.tf

# Initialises Terraform providers and sets their version numbers.

terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "3.90.0"
    }
  }

  required_version = "~> 1.7.2"
}

provider "azurerm" {
  features {}
}

## Terraform variables

- We are using the same Terraform variables as before
  - But have added an additional variable with the name `kubernetes_version` and the value `1.27.7`.
  - This variable is used to set the Kubernetes version to use in the file `kubernetes-cluster.tf`.

**Note! Make sure you change the value for the variable `app_name` to something unique!**

In [3]:
#!type variables.tf # use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.

variable "app_name" {
  default = "tsfn14g00"
}

variable "location" {
  default = "westeurope"
}

variable "kubernetes_version" {
  default = "1.27.7"
}

## Azure Resource Group

- We are using the same Azure Resoure Group as before.

In [4]:
#!type resource-group.tf # use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}

## Azure Container Registry

- We are using the same Azure Container Registry as before.
- We need this to store our Microservices' Images.
- When creating Pods in the Azure Kubernetes Service (AKS) cluster, the Images will be pulled from this Azure Container Registry.

In [5]:
#!type container-registry.tf # use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).

resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}

## Let's view the contents of the file `kubernetes-cluster.tf`

- Here we are defining an Azure Kubernetes Service (AKS), i.e. a Kubernetes cluster on Azure
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_kubernetes_cluster`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `kubernetes_cluster` is the name of the Azure resource (i.e. an Azure Kubernetes Service defined in the `azurerm` provider/plugin).
  - The first Argument sets the Azure Kubernetes Service's name
    - `name` is the argument's name
    - Its value is retrieved from the Terraform variable `app_name` (defined in the file `variables.tf`).
  - The second Argument sets the Azure Kubernetes Service's Location
    - `location` is the argument's name
    - Its value is retrieved from the Terraform variable `location` (defined in the file `variables.tf`).
  - The third Argument sets the Azure Resource Group in which the Azure Kubernetes Service will be created
    - `resource_group_name` is the argument's name
    - Its value is retrieved from the Terraform Expression `azurerm_resource_group.main.name`.
      - The `azurerm_resource_group.main` Block is defined in `resource-group.tf` as `resource "azurerm_resource_group" "main"`.
      - In this Block, there is an Argument with a name of `name` who's value is defined as `var.app_name`.
      - This is the value that is assigned to `resource_group_name`.
  - The fourth Argument sets the DNS Prefix for the Azure Kubernetes Service
    - `dns_prefix` is the argument's name
    - Its value is retrieved from the Terraform variable `app_name` (defined in the file `variables.tf`).
    - This parameter specifies the prefix to use for hostnames that are created for the DNS service in the cluster.
      - If not specified, a hostname is generated using a combination of the cluster's name and the resource group's name.
  - The fifth Argument sets version of Kubernetes to use for the cluster.
    - `kubernetes_version` is the argument's name
    - Its value is retrieved from the Terraform variable `kubernetes_version` (defined in the file `variables.tf`).
- The Block also contains two nested Blocks.
  - The first nested Block type is `default_node_pool`
    - An AKS cluster creates a Virtual Machine for each Node, and uses a "pool" for its Nodes, so that they can be scaled when required.
    - The first Argument sets the node pool's name
      - `name` is the argument's name
      - Its value is set to `"default"`.
    - The second Argument sets the initial number of Nodes in the pool
      - `node_count` is the argument's name
      - Its value is set to `1`.
    - The third Argument sets the size of the Virtual Machine used for each Node in the pool
      - `vm_size` is the argument's name
      - Its value is set to `"Standard_B2s"`.
      - For more information about Virtual Machine Sizes, see:
        - https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-b-series-burstable
  - The second nested Block type is `identity`
    - A AKS cluster needs an identity (service principle) which can be created manually or automatically.
    - The first and only Argument sets the AKS cluster's identity
      - `type` is the argument's name
      - Its value is set to `"SystemAssigned"`, which means Azure will create an identity (service principle) automatically.
      - For more information about Identities, see:
        - https://learn.microsoft.com/en-us/entra/identity/managed-identities-azure-resources/overview
- Lastly, we are defining an Azure Role Assignment, which automatically gives the AKS cluster access to the Container Registry.
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_role_assignment`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `role_assignment` is the name of the Azure resource (i.e. an Azure Role Assignment defined in the `azurerm` provider/plugin).
  - The first Argument sets the identity of the "principle" (object) that needs access to the Container Registry (a kubelet in the cluster)
    - `principal_id` is the argument's name
    - Its value is retrieved from the AKS resource `azurerm_kubernetes_cluster.main` we defined above, where
      - `kubelet_identity[0]` accesses the first item in this array property, and `object_id` is the identity of this kubelet.
  - The second Argument defines what type of access (authorization) the kubelet needs, which is to pull images from the Container Registry
    - `role_definition_name` is the argument's name
    - Its value is set to `"AcrPull"`, i.e. "Azure Container Registry Pull" which means the Kubelet is allowed to pull images.
  - The third Argument sets the scope (the identity of the object we are granting access to), which is the Azure Container Registry's identity
    - `scope` is the argument's name
    - Its value is set to `azurerm_container_registry.main.id`, where
      - `azurerm_container_registry.main` is the Azure Container Registry resource defined in the file `container-registry.tf`.
      - `id` is the property in that resource that contains the Azure Container Registry's identity.
  - The fourth Argument skips checking the kubelet's service principle (identity) in Azure Active Directory
    - `skip_service_principal_aad_check` is the argument's name.
    - Its value is set to `true`.
    - For more information see: https://registry.terraform.io/providers/hashicorp/azurerm/latest/docs/resources/role_assignment

In [6]:
#!type kubernetes-cluster.tf # use this on Windows
!cat kubernetes-cluster.tf

# Creates a managed Kubernetes cluster on Azure.
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name", "location" and "kubernetes_version" below are set from Terraform variables defined in the file "variables.tf".

resource "azurerm_kubernetes_cluster" "main" {
  name                = var.app_name
  location            = var.location
  resource_group_name = azurerm_resource_group.main.name
  dns_prefix          = var.app_name
  kubernetes_version  = var.kubernetes_version

  default_node_pool {
    name       = "default"
    node_count = 1
    vm_size    = "Standard_B2s"
  }

  # Instead of creating a service principle have the system figure this out.

  identity {
    type = "SystemAssigned"
  }
}

# Attach the Container Registry to t

## Let's view the contents of the file `network-watcher.tf`

- Microsoft requires every Virtual Network in Azure to have a Network Watcher.
  - Since an Azure Kubernetes Service (AKS) creates a Virtual Network, we also need a Network Watcher.
  - A Network Watcher is created automatically for a Virtual Network, but we are creating it manually here.
    - The only readson for creating it manually here, is so that a `terraform destroy` will delete it for us.
- Here we are defining two resources `azurerm_resource_group` and `azurerm_network_watcher`
  - `azurerm_resource_group` is a new Resource Group for the Network Watcher.
  - `azurerm_network_watcher` is the Network Watcher (which we place in the new Resource Group).
- For the Resource Group
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_resource_group"`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `resource_group` is the name of the Azure resource (i.e. an Azure Resource Group defined in the `azurerm` provider/plugin).
  - The second Block Label is `networkwatcher"` which is used to uniquely identity this resource in the Terraform files.
  - The first Argument sets the Azure Resource Groups's name
    - `name` is the argument's name
    - Its value is set to `NetworkWatcherRG`.
  - The second Argument sets the Azure Resource Group's Location
    - `location` is the argument's name
    - Its value is retrieved from the Terraform variable `location` (defined in the file `variables.tf`).
- For the Network Watcher
  - The Block Type is `resource`.
  - The first Block Label is `azurerm_network_watcher"`
    - `azurerm` is the name of the provider (i.e. the provider for Azure defined in the file `providers.tf`).
    - `network_watcher` is the name of the Azure resource (i.e. an Azure Network Watcher defined in the `azurerm` provider/plugin).
  - The second Block Label is `networkwatcher"` which is used to uniquely identity this resource in the Terraform files.
  - The first Argument sets the Azure Network Watcher's name
    - `name` is the argument's name
    - Its value is set to `NetworkWatcher_westeurope`.
  - The second Argument sets the Azure Network Wacther's Location
    - `location` is the argument's name
    - Its value is retrieved from the Terraform variable `location` (defined in the file `variables.tf`).
  - The third Argument places the Network Watcher in the Azure Resource Group we defined above
    - `resource_group_name` is the argument's name
    - Its value is retrieved from `azurerm_resource_group.networkwatcher.name`, where
      - `azurerm_resource_group.networkwatcher` refers to the Azure Resource Group we defined above.
      - `name` is the name property in the Azure Resource Group, which returns the name of the Resource Group.

In [7]:
#!type network-watcher.tf # use this on Windows
!cat network-watcher.tf

# Creates a Network Watcher on Azure.
# Note!
# - We create a "networkwatcher" (in its own Resource Group).
#   - This is required when a virtual network is created in Azure.
#     - An Azure Kubernetes Service (AKS) will create a virtual network.
#   - This is automatically created by Azure, but we explicitly create it here.
#     - The only reason we do this explicitly is so Terraform Destroy will automatically delete it for us.

resource "azurerm_resource_group" "networkwatcher" {
  name     = "NetworkWatcherRG"
  location = var.location
}

resource "azurerm_network_watcher" "networkwatcher" {
  name                = "NetworkWatcher_westeurope"
  location            = var.location
  resource_group_name = azurerm_resource_group.networkwatcher.name
}

## Initialize Terraform

- We initialize the Terraform Project as before.

In [8]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## Terraform Apply

- We apply the Terraform Project as before.

In [9]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the two Resource Groups defined in the Terraform project has been created.
- We also see an additional Resource Group `MC_tsfn14g00_tsfn14g00_westeurope` that is automatically created by Azure.
  - This Resource Group isn't included in the Terraform Project, but will automatically be destroyed by `terraform destroy`.

In [10]:
!az group list -o table

Name                               Location    Status
---------------------------------  ----------  ---------
tsfn14g00                          westeurope  Succeeded
NetworkWatcherRG                   westeurope  Succeeded
MC_tsfn14g00_tsfn14g00_westeurope  westeurope  Succeeded


## List Resources in Resource Group `tsfn14g00`

- The Azure CLI command `az resource list -n tsfn14g00 -o table` lists resources in Resource Group `tsfn14g00`.
- We see that the Resource Group contains the Azure Kubernetes Service and the Azure Container Registry.

In [11]:
!az resource list -n tsfn14g00 -o table

Name       ResourceGroup    Location    Type                                        Status
---------  ---------------  ----------  ------------------------------------------  --------
tsfn14g00  tsfn14g00        westeurope  Microsoft.ContainerRegistry/registries
tsfn14g00  tsfn14g00        westeurope  Microsoft.ContainerService/managedClusters


## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been created.

In [12]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-10T14:05:12Z  True


## List Repositories in Container Registry `tsfn14g00 `

- The Azure CLI command `az acr repository list -n tsfn14g00 --top 10 -o table` lists Repositories in a Azure Container Registry `tsfn14g00`.
  - The `-n` option is manditory and specifies the `NAME` of the Container Registry.
  - The `--top 10` limits the list to the first 10 Repositories (remove to see all Repositories).
- We see that Container Registry `tsfn14g00` doesn't contain any Repositories.

In [13]:
!az acr repository list -n tsfn14g00 --top 10 -o table

## Show Information about Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr show -n tsfn14g00 -o table` shows information about Container Registry `tsfn14g00`.
- It shows the Container Registry's `LOGIN SERVER` which is the URL to your Container Registry on Azure.
  - **This is the URL you would use to upload Docker Images to the Azure Container Registry.**

In [14]:
!az acr show -n tsfn14g00 -o table

# Let's store the LOGIN SERVER in a Python variable so we can use it later in this notebook
CONTAINER_REGISTRY_LOGIN_SERVER=!az acr show -n tsfn14g00 --query loginServer -o tsv
CONTAINER_REGISTRY_LOGIN_SERVER=CONTAINER_REGISTRY_LOGIN_SERVER[0]

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-10T14:05:12Z  True


## Show Credentials for Azure Container Registry `tsfn14g00`

- The Azure CLI command `az acr credential show -n tsfn14g00 -o table` shows credentials about Container Registry `tsfn14g00`.
- It shows the Container Registry's `USERNAME` and  `PASSWORD` to use to authenticate with your Azure Container Registry.
  - **This is the USERNAME and PASSWORD you would use to login to Docker to upload Images to the Azure Container Registry.**

In [15]:
!az acr credential show -n tsfn14g00 -o table

# Let's store the USERNAME and PASSWORD in Python variables so we can use them later in this notebook
CONTAINER_REGISTRY_USERNAME=!az acr credential show -n tsfn14g00 --query username -o tsv
CONTAINER_REGISTRY_USERNAME=CONTAINER_REGISTRY_USERNAME[0]
CONTAINER_REGISTRY_PASSWORD=!az acr credential show -n tsfn14g00 --query passwords[0].value -o tsv
CONTAINER_REGISTRY_PASSWORD=CONTAINER_REGISTRY_PASSWORD[0]

USERNAME    PASSWORD                                              PASSWORD2
----------  ----------------------------------------------------  ----------------------------------------------------
tsfn14g00   Z1lB6wzuValdlIy+kDVrpo9hbySz5E8r9FKxylOTu3+ACRAJDkyl  Wb0dHqFefw+utomWt8vpmelQa/ZlgD1Kd/RS/ucevA+ACRC3uwYO


## Login to Azure Container Registry via Docker

- Replace the variables below with your `LOGIN_SERVER`, `USERNAME` and `PASSWORD`.

In [17]:
!docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME -p $CONTAINER_REGISTRY_PASSWORD

# In Ubuntu with environment variables CONTAINER_REGISTRY_LOGIN_SERVER, CONTAINER_REGISTRY_USERNAME and CONTAINER_REGISTRY_USERNAME
#!echo $PASSWORD | docker login $LOGIN_SERVER -u $USERNAME --password-stdin  > /dev/null 2>&1

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


## Let's view the code in `video-streaming/src/index.js`

- The file contains code for creating a simple Node.js application, including the steps:
  1. Require the NPM package "express".
  2. Read the environment variable PORT to use as the Express web server's listen port.
  3. Creates an instance of an Express web server.
  4. Define a GET route for the "/video" path.
     1. Read information about the video file "videos/SampleVideo_1280x720_1mb.mp4".
     2. Write the HTTP response header including information about the file size in bytes and content type (video/mp4).
     3. Stream the file contents to the HTTP GET response.
  5. Start the Express web server listening on the port stored in environment variable PORT.
- As we see, this code produces a simple microservice that streams a video file from the file system to an HTTP GET response.

In [18]:
cat video-streaming/src/index.js

const express = require("express");
const fs = require("fs");

if (!process.env.PORT) {
    throw new Error("Please specify the port number for the HTTP server with the environment variable PORT.");
}

const PORT = process.env.PORT;

//
// Application entry point.
//
async function main() {

    const app = express();

    app.get("/video", async (req, res) => { // Route for streaming video.
        
        const videoPath = "./videos/SampleVideo_1280x720_1mb.mp4";
        const stats = await fs.promises.stat(videoPath);
    
        res.writeHead(200, {
            "Content-Length": stats.size,
            "Content-Type": "video/mp4",
        });

        fs.createReadStream(videoPath).pipe(res);
    });

    app.listen(PORT, () => {
        console.log("Microservice online.");
    });
}

main()
    .catch(err => {
        console.error("Microservice failed to start.");
        console.error(err && err.stack || err);
    });

## Initialize the Node.js project

- Let's initialise the Node.js project for the microservice by:
  - Running `npm init`.
  - Running `npm pkg` to configure `package.json`.
  - Running `npm install` to install the required NPM packages.

In [19]:
%%system
#rm -rf ./video-streaming/node_modules ./video-streaming/package.json ./video-streaming/package-lock.json
cd video-streaming
npm init -y
npm pkg set 'main'='./src/index.js'
npm pkg set 'scripts.start'='node ./src/index.js'
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js'
npm pkg delete 'scripts.test'
npm install --silent --save express@5.0.0-beta.1
npm install --silent --save-dev nodemon@3.0.3
cd ..

['Wrote to /home/patrick/projects/tsfn14/tsfn14/03_Azure_and_Terraform/04_kubernetes_cluster/video-streaming/package.json:',
 '',
 '{',
 '  "name": "video-streaming",',
 '  "version": "1.0.0",',
 '  "description": "",',
 '  "main": "index.js",',
 '  "scripts": {',
 '    "test": "echo \\"Error: no test specified\\" && exit 1"',
 '  },',
 '  "keywords": [],',
 '  "author": "",',
 '  "license": "ISC"',
 '}',
 '',
 '']

## Let's look at the Docker file used to build an image of the Node.js project

- Our image is based on the slimmed-down `alpine` version of `node:19.9.0`.
- The working directory in the image is `/usr/src/app`.
- The files `package.json` and `package-lock.json` are copied from the host computer to the image's working directory.
- The command `npm ci --omit-dev` is run in the image's working directory which installs non-dev NPM packages from `package.json`.
- The file `index.js` is copied from the host computer's `src` folder to the image's working directory's `src` folder.
- The file `SampleVideo_1280x720_1mb.mp4` is copied from the host computer's `videos` folder to the image's working directory's `videos` folder.
- The command `npm start` will be run in the working directory when an image container is started.

In [20]:
#!type video-streaming/Dockerfile # use this on Windows
!cat video-streaming/Dockerfile

FROM node:19.9.0-alpine

WORKDIR /usr/src/app
COPY package*.json ./
RUN npm ci --omit=dev
COPY ./src ./src
COPY ./videos ./videos

CMD npm start

## Build and Push a Docker Image to Azure Container Registry

- Here we are building an image of the Node.js application (microservice).
- We are tagging the image as `tsfn14g00.azurecr.io/video-streaming:1,`where:
  - `tsfn14g00.azurecr.io` is the URL (LOGIN SERVER) to our Container Registry.
  - `video-streaming` is the name of our image (repository).
  - `1` is the version of the image (tag).
- Then the image is pushed to the Azure Container Registry.
- Finally, the local image is removed from the host computer.

In [21]:
%%system

# Build Docker image with Nodejs Application
docker build -q -t {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1 -f ./video-streaming/Dockerfile ./video-streaming

# Push Docker Image to Azure Container Registry
docker push {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1

# Clean up
docker rmi {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1
docker images {CONTAINER_REGISTRY_LOGIN_SERVER}/video-streaming:1

['failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory',
 '',
 'DEPRECATED: The legacy builder is deprecated and will be removed in a future release.',
 '            Install the buildx component to build images with BuildKit:',
 '            https://docs.docker.com/go/buildx/',
 '',
 'sha256:8850cfb97104493c8041d2a667abae785b1fd1c40bc36d12c36aa2405c26fbe7',
 'The push refers to repository [tsfn14g00.azurecr.io/video-streaming]',
 'afa65e0fe51a: Preparing',
 '25cb75a35226: Preparing',
 'fcee9e1f0ab0: Preparing',
 'c37422ff2937: Preparing',
 '995d88721068: Preparing',
 'e394b5a78515: Preparing',
 '9b2d5af7f709: Preparing',
 'b240fe81adaa: Preparing',
 'bb01bd7e32b5: Preparing',
 'e394b5a78515: Waiting',
 '9b2d5af7f709: Waiting',
 'b240fe81adaa: Waiting',
 'bb01bd7e32b5: Waiting',
 'c37422ff2937: Pushed',
 '25cb75a35226: Pushed',
 '995d88721068: Pushed',
 'afa65e0fe51a: Pushed',
 'fcee9e1f0ab0: Pushed',
 'e394b5a78515: Pushed',
 '9

## Logout from the Azure Container Registry via Docker

In [23]:
!docker logout $CONTAINER_REGISTRY_LOGIN_SERVER

Removing login credentials for tsfn14g00.azurecr.io


## List Repositories in Container Registry `tsfn14g00 `

- We see that the Repository `video-streaming` has been created in Container Registry `tsfn14g00`.

In [24]:
!az acr repository list -n tsfn14g00 --top 10 -o table

Result
---------------
video-streaming


## Show Information about Repository `video-streaming`

- The Azure CLI command `az acr repository show -n tsfn14g00 --repository video-streaming -o table`
  - Shows information about Repository `video-streaming` in Container Registry `tsfn14g00`.
    - It contains images named `video-streaming` (ImageName).
    - It has a tag count of `1` (TagCount), i.e. currently there is only one tag for the `video-streaming` image.

In [25]:
!az acr repository show -n tsfn14g00 --repository video-streaming -o table

CreatedTime                   ImageName        LastUpdateTime                ManifestCount    Registry              TagCount
----------------------------  ---------------  ----------------------------  ---------------  --------------------  ----------
2024-02-10T14:12:06.6669439Z  video-streaming  2024-02-10T14:12:06.7670932Z  1                tsfn14g00.azurecr.io  1


## List Tags in Repository `video-streaming`

- The Azure CLI command `az acr repository show-tags -n tsfn14g00 --repository video-streaming --top 10 -o table`:
  - Lists the tags in Repository `video-streaming` in Container Registry `tsfn14g00`.
    - Currently there is only one tag.
    - The tag has the value `1`.

In [26]:
!az acr repository show-tags -n tsfn14g00 --repository video-streaming --top 10 -o table

Result
--------
1


## Show Information about Image `video-streaming:1`

- The Azure CLI command `az acr repository show -n tsfn14g00 --image video-streaming:1 -o table`:
  - Shows information about image `video-streaming:1` in Container Registry `tsfn14g00`.
    - The information includes the Digest for the image.

In [27]:
!az acr repository show -n tsfn14g00 --image video-streaming:1 -o table

CreatedTime                   Digest                                                                   LastUpdateTime                Name    Signed
----------------------------  -----------------------------------------------------------------------  ----------------------------  ------  --------
2024-02-10T14:12:06.7993853Z  sha256:07dc63ad37c3031d10b0254e133c41ee93402f3bad2208cafe86dcea1928a5c9  2024-02-10T14:12:06.7993853Z  1       False


## Show Azure Container Registry Usage

- The command `az acr show-usage -n tsfn14g00 -o table` shows the usage of Container Registry `tsfn14g00`.
  - We can see the maximum number of allowed bytes in the `LIMIT` column (first row).
  - We can see the current number of bytes in the `CURRENT VALUE` column (first row).

In [28]:
!az acr show-usage -n tsfn14g00 -o table

NAME       LIMIT        CURRENT VALUE    UNIT
---------  -----------  ---------------  ------
Size       10737418240  56544784         Bytes
Webhooks   2            0                Count
ScopeMaps  100          0                Count
Tokens     100          0                Count


## List Azure Kubernetes Services

- The Azure CLI command `az aks list -o table` lists all Azure Kubernetes Services in Azure.
- We see that the Azure Kubernetes Service (AKS) defined in the Terraform project has been created.

In [29]:
!az aks list -o table

Name       Location    ResourceGroup    KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------  ----------  ---------------  -------------------  --------------------------  -------------------  -------------------------------------------
tsfn14g00  westeurope  tsfn14g00        1.27.7               1.27.7                      Succeeded            tsfn14g00-9tvl5tf4.hcp.westeurope.azmk8s.io


## Add Azure Kubernetes Cluster Info. to Local Kubectl Config File

- The Azure CLI command `az aks get-credentials --name tsfn14g00 --resource-group tsfn14g00` will:
  - Get the credentials from the Azure Kubernetes Service `tsfn14g00` in Resource Group `tsfn14g00`.
  - Add the credentials to the 'kubectl' CLI tool's `config` file on the host machine.
    - The 'kubectl' CLI tool's `config` file is located at  `~/.kube/config` on Linux/macOs.
    - The 'kubectl' CLI tool's `config` file is located at  `%USERPROFILE%\.kube\config` on Windows.

In [30]:
!az aks get-credentials --name tsfn14g00 --resource-group tsfn14g00

# The command below attaches the Azure Kubernetes Cluster to the Azure Container Registry so that
# the Azure Kubernetes Cluster can pull images from the Azure Container Registry, but we don't
# have to do this explicitly here, since we do it in the Terraform file "kubernetes-cluster.tf".
#az aks update -n tsfn14g00 -g tsfn14g00 --attach-acr tsfn14g00 -o table

Merged "tsfn14g00" as current context in /home/patrick/.kube/config


## Ensure the Kubectl Context is set to the Azure Kubernetes Cluster

- The kubectl CLI command:
  - `kubectl config get-contexts` lists all contexts in kubectl's config file.
  - `kubectl config use-context [ContextName]` sets the current context to `[ContextName]` in kubectl's config file.
  - `kubectl config current-context` returns the current context set in kubectl's config file.
  - `kubectl config view` lists the contexts of kubectl's config file.
- Here we just want to make sure kubectl's current context is set to the Azure Kubernetes Service's context.
  - We see that the current context is `tsfn14g00` (or whatever you set the `app_name` Terraform variable to).

In [31]:
#!kubectl config get-contexts
#!kubectl config use-context tsfn14g00
!kubectl config current-context
#!kubectl config view

tsfn14g00


## List Deployments, Pods and Services

- The kubectl command:
  - `kubectl get deployments -o wide` lists all Deployments in the kubernetes cluster.
  - `kubectl get pods -o wide` lists all Pods in the kubernetes cluster.
  - `kubectl get services -o wide` lists all Services in the kubernetes cluster.
- Notice we have:
  - Not Deployments or Pods in the Azure Kubernetes Cluster.
  - One ClusterIP Service in the Azure Kubernetes Cluster already defined before deploying any resources ourselves.

In [32]:
!kubectl get deployments -o wide
!kubectl get pods -o wide
!kubectl get services -o wide

No resources found in default namespace.
No resources found in default namespace.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE     SELECTOR
kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   7m12s   <none>


## Lets' look at the code in the file `manifests/deployment.yaml`

- We see two kubenetes resources defined in the file:
  - `kind: Deployment` and `kind: Service`.
  - Notice how you can define multiple resources in a YAML file by separating them with three dashes `---`.
- Deployment:
  - The Deployment's `name` is `video-streaming`, sets `replicas` to `1`, and its `matchLabels` contains `app: video-streaming`.
  - The Pod template's `labels` contains `app: video-streaming` and its settings under `containers` are defined as below:
    - The container's `name` is `video-streaming`, and it pulls the `image` called `tsfn14g00.azurecr.io/video-streaming:1`
      - `tsfn14g00.azurecr.io` is the URL to the Azure Container Registry.
      - `video-streaming:1` is the name of the image, including the image's version (tag).
    - The container sets one environment variable named `PORT` with the value `4000`.
    - The container also `requests` `128m` of the node's `cpu` and `128Mi` of the node's `memory.`
    - The container also `limits` the use to `256m` of the node's `cpu` and `256i` o the node's `memory.`
- Service:
  - The Service's `name` is `video-streaming`, sets it's `type` to `LoadBalancer`, and its `matchLabels` contains `app: video-streaming`.
  - The Service is listening on `port` `80` and is redirecting traffic to `targetPort` `4000`.

In [33]:
#!type manifests/deployment.yaml # use this on Windows
!cat manifests/deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: video-streaming
spec:
  replicas: 1
  selector:
    matchLabels:
      app: video-streaming
  template:
    metadata:
      labels:
        app: video-streaming
    spec:
      containers: 
      - name: video-streaming
        image: tsfn14g00.azurecr.io/video-streaming:1
        imagePullPolicy: IfNotPresent
        env:
        - name: PORT
          value: "4000"
        resources:
          requests:
            cpu: 128m
            memory: 128Mi
          limits:
            cpu: 256m
            memory: 256Mi
---
apiVersion: v1
kind: Service
metadata:
  name: video-streaming
spec:
  selector:
    app: video-streaming
  type: LoadBalancer
  ports:
    - protocol: TCP
      port: 80
      targetPort: 4000

## Apply Deployment to the Azure Kubernets Cluster

- The kubectl command `kubectl apply -f manifests/deployment.yaml` will:
  -  Apply the resource definitions in the file `manifests/deployment.yaml` to the kubernetes cluster.

In [34]:
!kubectl apply -f manifests/deployment.yaml

deployment.apps/video-streaming created
service/video-streaming created


## List Deployments, Pods and Services

- The kubectl command:
  - `kubectl get deployments -o wide` lists all Deployments in the kubernetes cluster.
  - `kubectl get pods -o wide` lists all Pods in the kubernetes cluster.
  - `kubectl get services -o wide` lists all Services in the kubernetes cluster.
- We see that the Deployment, with associated Pods (1 replica), and Service defined in the applied manifest have been created.
- Furhermote, we see the Load Balancer's `EXTERNAL-IP` (public IP) which is the IP address you use to access your Service externally.
  - **This is the IP address you would use to access your Service publically over the internet, e.g. http://EXTERNAL-IP**

In [35]:
!kubectl get deployments
!kubectl get pods
!kubectl get services

# Let's store the Load Balancer's EXTERNAL-IP (public IP) in a Python variable so we can use it later in this notebook
LOADBALANCER_PUBLIC_IP=!kubectl get service video-streaming -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
LOADBALANCER_PUBLIC_IP=LOADBALANCER_PUBLIC_IP[0]

NAME              READY   UP-TO-DATE   AVAILABLE   AGE
video-streaming   1/1     1            1           36s
NAME                               READY   STATUS    RESTARTS   AGE
video-streaming-864b6976bf-q2fz9   1/1     Running   0          36s
NAME              TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes        ClusterIP      10.0.0.1       <none>        443/TCP        7m56s
video-streaming   LoadBalancer   10.0.101.109   20.82.72.18   80:32258/TCP   36s


## Test the Microservice Application in the Kubernetes Cluster

- Open a browser and enter the URL `http://EXTERNAL-IP/video`
  - This sends an HTTP GET request to the Microservice's GET route for the `/video` path.
  - Streams the video `SampleVideo_1280x720_1mb.mp4` stored in the Microservice's container's file system to the browser.

In [36]:
#!firefox http://{LOADBALANCER_PUBLIC_IP}/video

## Delete Deployment in the Azure Kubernets Cluster

- The kubectl command `kubectl delete -f manifests/deployment.yaml` will:
  - Delete the resources defined in the file `manifests/deployment.yaml` from the kubernetes cluster.

In [37]:
!kubectl delete -f manifests/deployment.yaml

deployment.apps "video-streaming" deleted
service "video-streaming" deleted


## List Deployments, Pods and Services

- The kubectl command:
  - `kubectl get deployments -o wide` lists all Deployments in the kubernetes cluster.
  - `kubectl get pods -o wide` lists all Pods in the kubernetes cluster.
  - `kubectl get services -o wide` lists all Services in the kubernetes cluster.
- We see that the Deployment, with associated Pods (1 replica), and Service defined in the manifest file have been deleted.

In [38]:
!kubectl get deployments
!kubectl get pods
!kubectl get services

No resources found in default namespace.
No resources found in default namespace.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   9m15s


## Ensure the Kubectl Context is set to the Minikube Cluster

**Note!**

- Replace `minikube` below with the name of your local (development) kubernetes cluster (mine is called `minikube`).
- You can list all contexts defined in kubectls config file using the command `kubectl config get-contexts`.
- If you don't have another context in your config file:
  - ue the kubectl command `kubectl config unset current-context` instead of `kubectl config use-context minikube`.

In [39]:
#!kubectl config get-contexts
!kubectl config use-context minikube
#!kubectl config unset current-context
!kubectl config current-context

Switched to context "minikube".
minikube


## Remove Azure Kubernetes Cluster Info. from Local Kubectl Config File

- The kubectl command:
  - `kubectl config delete-cluster tsfn14g00` deletes the cluster called `tsfn14g00` from kubectl's config file.
  - `kubectl config delete-context tsfn14g00` deletes the context celled `tsfn14g00` from kubectl's config file.
  - `kubectl config delete-user tsfn14g00` deletes the user called `tsfn14g00` from kubectl's config file.
  - `kubectl config view` lists the contents of kubectl's config file.
- kubectl's config file is located in your host machine's file system.
    - The config file is located at `~/.kube/config` on Linux/macOs.
    - The config file is located at `%USERPROFILE%\.kube\config` on Windows.

**Note**

- Reaplce `tsfn14g00` with the value you chose for the Terraform variable `app_name` (in the file `variables.tf`).

In [40]:
!kubectl config delete-cluster tsfn14g00
!kubectl config delete-context tsfn14g00
!kubectl config delete-user clusterUser_tsfn14g00_tsfn14g00
#!kubectl config view

deleted cluster tsfn14g00 from /home/patrick/.kube/config
deleted context tsfn14g00 from /home/patrick/.kube/config
deleted user clusterUser_tsfn14g00_tsfn14g00 from /home/patrick/.kube/config


## Terraform 

- We destroy the Terraform Project as before.

In [41]:
!terraform destroy -auto-approve

azurerm_resource_group.networkwatcher: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/NetworkWatcherRG]
azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]
azurerm_kubernetes_cluster.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerService/managedClusters/tsfn14g00]
azurerm_network_watcher.networkwatcher: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/NetworkWatcherRG/providers/Microsoft.Network/networkWatchers/NetworkWatcher_westeurope]
azurerm_role_assignment.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/reso

## List Azure Kubernetes Services

- The Azure CLI command `az aks list -o table` lists all Azure Kubernetes Services in Azure.
- We see that the Azure Kubernetes Service (AKS) defined in the Terraform project has been destroyed.

In [42]:
!az aks list -o table

## List Azure Container Registries

- The Azure CLI command `az acr list -o table` lists all Container Registries in Azure.
- We see that the Container Registry defined in the Terraform project has been destroyed.

In [43]:
!az acr list -o table

## List Azure Resource Groups

- The Azure CLI command `az group list -o table` lists all Resource Groups in Azure.
- We see that the two Resource Groups defined in the Terraform project has been destroyed.

In [44]:
!az group list -o table